## Utility Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

%matplotlib widget

def draw(ax, verts, faces=[], color=[]):
    """
    Draw the object defined by vertices and faces.
    """

    if verts.ndim == 1:
        if color:
            ax.scatter3D(verts[0], verts[1], verts[2], c=color)
        else:
            ax.scatter3D(verts[0], verts[1], verts[2], c=color)
    else:
        if color:
            ax.scatter3D(verts[:, 0], verts[:, 1], verts[:, 2], c=color)
        else:
            ax.scatter3D(verts[:, 0], verts[:, 1], verts[:, 2])

    if faces:
        ax.add_collection3d(Poly3DCollection(faces,
                                             facecolors='blue',
                                             linewidths=1,
                                             edgecolor='b',
                                             alpha=0.25))

def build_projection_matrix(focal_length=1.0, reverse=False):
    """
    Builds a simple projection matrix given the distance of the viewing plane
    from the camera.
    
    Args:
        focal_length -- distance from lens to sensor (default 1.0)
        reverse -- determines projection to virtual plane (False) or sensor plane (True)
    """
    
    proj = np.array([[focal_length, 0, 0, 0],
                     [0, focal_length, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 1, 0]])
    
    if reverse is True:
        proj[2:, 2] *= -1
        
    return proj


def draw_camera(ax, camera_pos, V, d=1.0, draw_physical=False):
    """Draws a camera at `camera_pos` with a view plane defined by transform `T` and distance `d`.
    
    Args:
        ax -- matplotlib axis
        camera_pos -- camera's position in world space
        V -- camera transformation matrix (virtual)
        d -- distance of viewing plane from camera (default 1.0)
        draw_physical -- draw physical sensor plane (default False)
    """
    # Viewing Plane
    view_plane = np.array([[-1, -1, 0],
                           [1, -1, 0],
                           [1, 1, 0],
                           [-1, 1, 0]])

    view_plane_virtual = transform_points(view_plane, np.linalg.inv(V))
    
    forward_vector = V[2, :-1]
    
    M = np.eye(4)
    f_vec = forward_vector / np.linalg.norm(forward_vector)
    M[:-1, -1] = f_vec.T * d
    view_plane_virtual = transform_points(view_plane_virtual, M)
    view_faces = [[view_plane_virtual[0], view_plane_virtual[1], view_plane_virtual[2], view_plane_virtual[3]]]
    
    draw(ax, camera_pos)
    draw(ax, view_plane_virtual, view_faces)
    for i in range(len(view_plane)):
        ax.plot([camera_pos[0], view_plane_virtual[i, 0]],
                [camera_pos[1], view_plane_virtual[i, 1]],
                [camera_pos[2], view_plane_virtual[i, 2]], c='k')
        
    if draw_physical:
        view_plane_physical = transform_points(view_plane, np.linalg.inv(V))

        forward_vector = -V[2, :-1]
        M = np.eye(4)
        f_vec = forward_vector / np.linalg.norm(forward_vector)
        M[:-1, -1] = f_vec.T * d
        view_plane_physical = transform_points(view_plane_physical, M)
        view_faces = [[view_plane_physical[0], view_plane_physical[1], view_plane_physical[2], view_plane_physical[3]]]

        draw(ax, view_plane_physical, view_faces)
        for i in range(len(view_plane)):
            ax.plot([camera_pos[0], view_plane_physical[i, 0]],
                    [camera_pos[1], view_plane_physical[i, 1]],
                    [camera_pos[2], view_plane_physical[i, 2]], c='k')
            
            
def compute_projected_points3d(points, camera_matrix, projection_matrix, focal_length, reverse=False):
    """Computes 3D points on the image plane as seen from world space.
    
    Args:
        points -- 3D world points to be drawn
        camera_matrix -- world-to-camera matrix
        projection_matrix -- camera's projection matrix
        focal_length -- focal length of camera
        reverse -- project to virtual (False) or physical (True)
    """
    
    forward_vector = camera_matrix[2, :-1]
            
    M = np.eye(4)
    f_vec = forward_vector / np.linalg.norm(forward_vector)
    M[:-1, -1] = f_vec.T * (focal_length - 1)
    
    if reverse:
        M[:-1, -1] = -2 * f_vec.T - M[:-1, -1]        
        
    return transform_points(points, M @ np.linalg.inv(camera_matrix) @ projection_matrix @ camera_matrix)
    

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm


def look_at(src, to, up):
    """Creates a camera matrix located at `src` looking towards `to`."""
    forward = normalize(src - to)
    right = np.cross(normalize(up), forward)
    up = np.cross(forward, right)

    forward = -forward

    camera_matrix = np.array([
        [right[0], right[1], right[2], -np.dot(right, src)],
        [up[0], up[1], up[2], -np.dot(up, src)],
        [forward[0], forward[1], forward[2], -np.dot(forward, src)],
        [0, 0, 0, 1]
    ])

    return camera_matrix


def transform_point(point, T):
    proj_point = T @ np.append(point, 1)
    proj_point /= proj_point[3]

    return proj_point[:3]


def transform_points(points, T):
    homogeneous_points = np.hstack((points, np.ones((len(points), 1))))
    proj_points = T @ homogeneous_points.T
    proj_points /= proj_points[3, :]

    return proj_points[:3, :].T

## Camera Class

In [ ]:
class Camera:
    def __init__(self, position, target, up, focal_length=1.0):
        """
        Initializes a camera object with position, target, and up vector.
        Args:
            position -- camera position in world space
            target -- target point in world space
            up -- up vector in world space
            focal_length -- focal length of the camera (default 1.0)
        """
        self.focal_length = focal_length
        self.position = position
        self.target = target
        self.up = up
        self.camera_matrix = look_at(position, target, up)
        self.projection_matrix = build_projection_matrix(focal_length)
        self.view_matrix = np.linalg.inv(self.camera_matrix)

    def project(self, points):
        """
        Projects 3D points onto the image plane.
        Args:
            points -- 3D points to be projected
        Returns:
            Projected 2D points
        """
        return compute_projected_points3d(points, self.camera_matrix, self.projection_matrix, self.focal_length)

    def draw(self, ax):
        """
        Draws the camera and its view plane in the given axis.
        Args:
            ax -- matplotlib axis
        """
        draw_camera(ax, self.position, self.camera_matrix, self.focal_length)
        ax.set_title('Camera View')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_xlim([-2, 5])
        ax.set_ylim([-2, 10])
        ax.set_zlim([-3, 4])
        ax.view_init()
        ax.grid(True)
        ax.set_box_aspect([1, 1, 1])

In [ ]:
plt.close()

# Construct two cameras in a simple stereo setup offset by 3 units
camera1_pos = np.array([0, 0, 0])
camera2_pos = np.array([3, 0, 0])
camera1_target = np.array([0, 1, 0])
camera1_up = np.array([1, 0, 0])
camera2_target = np.array([3, 1, 0])
camera2_up = np.array([1, 0, 0])
camera1 = Camera(camera1_pos, camera1_target, camera1_up)
camera2 = Camera(camera2_pos, camera2_target, camera2_up)

# Create a 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Draw a point
point = np.array([[1.5, 7, 0]])
draw(ax, point, color='r')

# Draw the point in the camera's view
point_camera1 = camera1.project(point)
point_camera2 = camera2.project(point)

# Draw the projected points
draw(ax, point_camera1, color='g')
draw(ax, point_camera2, color='g')

# Draw the cameras
camera1.draw(ax)
camera2.draw(ax)

# Draw a line from the cameras to the world point
ax.plot([camera1_pos[0], point[0, 0]],
        [camera1_pos[1], point[0, 1]],
        [camera1_pos[2], point[0, 2]], c='k')
ax.plot([camera2_pos[0], point[0, 0]],
        [camera2_pos[1], point[0, 1]],
        [camera2_pos[2], point[0, 2]], c='k')


plt.show()